In [17]:
expt_name="mnist"
fname="/home/pawan/Dropbox/SymAE/"+expt_name
fnametest="/home/pawan/Dropbox/SymAE/test"+expt_name

In [18]:
%run ../core.py

In [19]:
dat=h5py.File(fname+".h5", 'r')
# dattest=h5py.File(fnametest+".h5", 'r')
Xo_train=np.array(dat["data"])
# Xo_test=np.array(dattest["data"])

# Xo_train=np.expand_dims(Xo_train,[3])
#Xo_test=np.expand_dims(Xo_test,[3])
    
nsamp, ntau, n1, n2, nfilt=Xo_train.shape
with open(str(fname+'.json')) as f:
    labels_train = json.load(f)
labels_train=[labels_train[str(i)] for i in range(1,nsamp+1)]
#with open(str(fnametest+'.json')) as f:
#    labels_test = json.load(f)
#labels_test=[labels_test[str(i)] for i in range(1,3)]


print("shape of Xo_train",np.shape(Xo_train))
#print("shape of Xo_test",np.shape(Xo_test))
print("number of samples Xo_train\t", nsamp)
print("number of instances in each datapoint Xo_train\t", ntau)
print("number of samples in each channel\t", (n1,n2))

shape of Xo_train (10000, 20, 28, 28, 1)
number of samples Xo_train	 10000
number of instances in each datapoint Xo_train	 20
number of samples in each channel	 (28, 28)


In [20]:

# Xo_train, labels_train, Xo_test, labels_test=read_datapoints(fname, fnametest)
# nsamp, ntau,n1,n2,nfilt=Xo_train.shape


# In[5]:


from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# Xo_train, Xo_test=train_test_split(Xo, test_size=0.1, shuffle=True)
# Xo_train, labels_train=shuffle(Xo_train, labels_train, n_samples=9000)
Xo_train, Xo_test, labels_train, labels_test=train_test_split(Xo_train, labels_train, test_size=0.1, shuffle=True)
nsamp, ntau, n1,n2,nfilt =Xo_train.shape



In [21]:
%run ../symae_core.py

Num GPUs Available:  1
2.4.1


In [22]:

# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])
# strategy = tf.distribute.MirroredStrategy()
# strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1","/gpu:2","/gpu:3","/gpu:4","/gpu:5","/gpu:6","gpu:7"])
strategy = tf.distribute.OneDeviceStrategy("/gpu:0")
# strategy = tf.distribute.get_strategy()
# strategy = tf.distribute.MirroredStrategy(devices=["gpu:7"])
# strategy = tf.distribute.experimental.CentralStorageStrategy() 
#%% model
with strategy.scope():
    
    
    encoder_input=tfk.Input(shape=(ntau,n1,n2,1), dtype='float32', name='encoder_input')
    
#     downsampler=Downsampler([2,25],8,2)
#     downsampler=DownsamplerDense(200,nt)
#     xd=downsampler(encoder_input)

#     downsampler.summary(line_length=120)
#     tf.keras.utils.plot_model( downsampler, to_file='encoderdown.png', show_shapes=True, 4
    kernel_size=(5,5)
    nfilt=64
    
    nz0=2
    symencoder1=SymmetricEncoder(kernel_size,nfilt,[2,2,2,1],[2,2,2,1],nz0)
    xz0=symencoder1(encoder_input)
    
    z0=tfkl.Flatten()(xz0)
    
    nz0=z0.get_shape()[1]
    
    report(symencoder1.model(encoder_input),"symencoder")
    symencoder=tf.keras.Model(encoder_input, z0, name="symencoder")
    
    
    nzi=5
    nencoder1=NuisanceEncoder(kernel_size,nfilt,[2,2,2,1],[2,2,2,1],nfilt,nzi)
    
    xzi=nencoder1(encoder_input)

    zi=tfkl.Flatten()(xzi)

    report(nencoder1.model(encoder_input),"nencoder")
    nencoder=tf.keras.Model(encoder_input, zi, name="nencoder")
    
    decoder_input = tf.keras.Input(shape=(nzi*ntau+nz0), name='latentcode')
    
    distzsym=DistributeZsym(ntau, nz0, nzi)
    xdhat=distzsym(decoder_input)
    
    report(distzsym.model(decoder_input),"distzsym")

    mixer=Mixer(kernel_size,nfilt,(2,2),n1,n2)
    xhat=mixer(xdhat)

#     upsampler=Upsampler([2,25],8,2)
#     upsampler=UpsamplerDense(nt,200)
#     xhat=upsampler(xdhat)
    
    znuisance=nencoder(encoder_input)
    zsym=symencoder(encoder_input)
    
    
    report(mixer.model(tfk.Input(shape=(ntau,nzi+nz0))),"mixer")
    
#     report(upsampler,"upsampler")
    
    

Model: "model_4"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
encoder_input (InputLayer)                                                                [(None, 20, 28, 28, 1)]                                                         0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
time_distributed_33 (TimeDistributed)                                                     (None, 20, 28, 28, 64)                                                          1664     

In [23]:

with strategy.scope():
    latentcat=LatentCat(0.5)
    
    encoder=tf.keras.Model(encoder_input, latentcat(zsym,znuisance), name="encoder")

    decoder=tf.keras.Model(decoder_input, xhat, name="decoder")
    report(decoder, "decoder")
    
    model=tf.keras.Model(encoder_input, decoder(latentcat(zsym,znuisance)) , name='autoencoder')
    report(model, "autoencoder")

model.compile(optimizer='adam', loss='mse')

Model: "decoder"
________________________________________________________________________________________________________________________________________________________________________________________________________
Layer (type)                                                                              Output Shape                                                                    Param #                       
latentcode (InputLayer)                                                                   [(None, 102)]                                                                   0                             
________________________________________________________________________________________________________________________________________________________________________________________________________
distribute_zsym_1 (DistributeZsym)                                                        (None, 20, 7)                                                                   0        

In [24]:


batch_size=8
ds_train=tf.data.Dataset.from_tensor_slices((Xo_train, Xo_train))
# ds_train = tf.data.TFRecordDataset(filenames=["Xo.tfrecords"])
#   print(ds_train)
ds_train = ds_train.shuffle(nsamp,reshuffle_each_iteration=True) 
ds_train=ds_train.batch(batch_size)
# ds_train = ds_train.cache()
# ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

ds_test=tf.data.Dataset.from_tensor_slices((Xo_test, Xo_test))
ds_test=ds_test.batch(batch_size)
# ds_test = ds_test.cache()
# ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)


In [25]:


options = tf.data.Options()
# options.experimental_optimization.noop_elimination = True
# options.experimental_optimization.map_vectorization.enabled = True
# options.experimental_threading.max_intra_op_parallelism = 1 # reduces CPU usage?
# options.experimental_optimization.apply_default_optimizations = True
ds_train = ds_train.with_options(options)
ds_test = ds_test.with_options(options)

In [26]:
#cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=base_dir + expt_name + '_TFmodel_save', save_weights_only=True, verbose=1)
#model.fit(Xr,Xr,epochs=10, batch_size=batch_size,  callbacks=[cp_callback, generate_and_save_images(), history], validation_split=0.1, shuffle=True)
#model.fit(Xr,Xr,epochs=10, batch_size=batch_size,  callbacks=[generate_and_save_images(), history], validation_split=0.1, shuffle=True)
# model.fit(Xo,Xo,epochs=100, batch_size=batch_size, callbacks=[tbCallBack], shuffle=True, validation_split=0.05)
history=model.fit(ds_train,epochs=500,  shuffle=True, validation_data=ds_test)
# model.fit(ds_train,epochs=40 )
# model.fit(Xo,Xo,epochs=1,  batch_size=4)#, callbacks=[tbCallBack])

Epoch 1/500
1125/1125 [==============================] - 30s 25ms/step - loss: 0.6585 - val_loss: 0.3879
Epoch 2/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.4342 - val_loss: 0.3366
Epoch 3/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.4039 - val_loss: 0.3195
Epoch 4/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.3874 - val_loss: 0.3025
Epoch 5/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.3776 - val_loss: 0.3023
Epoch 6/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.3694 - val_loss: 0.2987
Epoch 7/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.3661 - val_loss: 0.2931
Epoch 8/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.3603 - val_loss: 0.2916
Epoch 9/500
1125/1125 [==============================] - 28s 25ms/step - loss: 0.3540 - val_loss: 0.2791
Epoch 10/500
1125/1125 [==============================]

KeyboardInterrupt: 

In [30]:
%run ../plots.py

In [35]:
plot_mnist(Xo_train, labels_train,symencoder, nencoder, decoder)


test_indices7986,1606
01-02-2021_11/3_7986-true-0_itrue.png
01-02-2021_11/7_1606-true-0_jtrue.png
01-02-2021_11/3_7986-7986-0_ii.png
01-02-2021_11/3_7986-1606-0_ij.png
01-02-2021_11/7_1606-1606-0_jj.png
01-02-2021_11/7_1606-7986-0_ji.png
01-02-2021_11/3_7986-true-1_itrue.png
01-02-2021_11/7_1606-true-1_jtrue.png
01-02-2021_11/3_7986-7986-1_ii.png
01-02-2021_11/3_7986-1606-1_ij.png
01-02-2021_11/7_1606-1606-1_jj.png
01-02-2021_11/7_1606-7986-1_ji.png
01-02-2021_11/3_7986-true-2_itrue.png
01-02-2021_11/7_1606-true-2_jtrue.png
01-02-2021_11/3_7986-7986-2_ii.png
01-02-2021_11/3_7986-1606-2_ij.png
01-02-2021_11/7_1606-1606-2_jj.png
01-02-2021_11/7_1606-7986-2_ji.png
01-02-2021_11/3_7986-true-3_itrue.png
01-02-2021_11/7_1606-true-3_jtrue.png
01-02-2021_11/3_7986-7986-3_ii.png
01-02-2021_11/3_7986-1606-3_ij.png
01-02-2021_11/7_1606-1606-3_jj.png
01-02-2021_11/7_1606-7986-3_ji.png
01-02-2021_11/3_7986-true-4_itrue.png
01-02-2021_11/7_1606-true-4_jtrue.png
01-02-2021_11/3_7986-7986-4_ii.png
01-

In [59]:
print("Previous Dropout "+str(model.layers[2].layers[-1].rate))
model.save_weights("temp_weights")
new_dropout=0.5
model.layers[2].layers[-1].rate=new_dropout
model = tfk.models.clone_model(model)
model.compile(optimizer='adam', loss='mse')
model.load_weights("temp_weights")
print("New Dropout "+str(model.layers[2].layers[-2].rate))

AttributeError: 'Flatten' object has no attribute 'rate'

In [ ]:


labelindices={'base0': [], 'moni1' : [], 'moni2' : []}
for (i,label) in enumerate(labels):
    labelindices[label[0][0]].append(i)


# In[281]:


labels[labelindices['moni2']][0][0]


# # Visualize Latent Space

# In[ ]:





# In[80]:


# visualizing latent space

"""
fig=plt.figure(dpi=100, figsize=(15,8))
for label in ['100', '200', '300', '400', '500']:
    i = labelindices[label][0:1]
    
    with tf.device('/cpu:0'):
        latentsr=encoderr(Xo[i])
    
    

    """

with tf.device('/gpu'):
    test_large_indices = [np.random.randint(1, nsamp) for i in range(0,64)]
    latentsT=nencoder.predict(Xo_train                           [test_large_indices])
    latentsr=symencoder.predict(Xo_train[test_large_indices])
    #print(np.shape(latentsr))
    fig, axs = plt.subplots(2, 2, dpi=100, figsize=(15,8))
    a=axs[0,0].imshow(latentsr, aspect=1)
    #for l in latentsr:
    #    a=axs[0,0].plot(l)
    axs[0,0].set_title('G')
    plt.colorbar(a, ax=axs[0,0]) 

    #a=axs[1,0].imshow(tf.exp(latentsr[1]), aspect=1)
    #axs[1,0].set_title('sigma encoderr output for samples')
    #plt.colorbar(a, ax=axs[1,0]) 

    b=axs[0,1].imshow(latentsT, aspect=1)
    axs[0,1].set_title('Z')
    plt.colorbar(b, ax=axs[0,1]) 
    
plt.gcf()
#b=axs[1,1].imshow(tf.exp(latentsT[1]), aspect=1)
#axs[1,1].set_title('sigma encoderT output for samples')
#plt.colorbar(b, ax=axs[1,1]) 


# In[67]:


plt.imshow(latentsT)


# In[68]:


with tf.device('/gpu'):
    latentsr={'base0': [], 'moni1':[],'moni2':[]}
    for label in ['base0', 'moni1', 'moni2']:
        i = labelindices[label][1:20]
        with tf.device('/gpu:0'):
            latentsr[label]=symencoder.predict(Xo_train[i])


# In[69]:


fig, ax = plt.subplots(2,5,dpi=500, figsize=(3,2))
markers=['r', 'b', 'c', 'g', 'm']
subs=['(a)', '(b)', '(c)', '(d)', '(e)']
for (i,label) in enumerate(['base0', 'moni1', 'moni2', 'base0', 'base0']):
    lmean=np.mean(latentsr[label], axis=0)
    #ax[i].plot(lmean, color='k', linewidth=0.1)
    ax[1,i].imshow(latentsr[label].T)
    # Hide grid lines
    ax[1,i].grid(False)
#     Hide axes ticks
    ax[1,i].set_xticks([])
    ax[1,i].set_yticks([])
    ax[1,i].set_xlabel('$\tau$')
    ax[1,i].set_ylabel('$Z_g$')
    
    
    
    ax[0,i].imshow(vp[label].T, cmap='RdBu')
    ax[0,i].axis('off')
    ax[0,i].set_xlabel('$x$')
    ax[0,i].set_ylabel('$z$')
    ax[0,i].set_title(subs[i]+'$\epsilon=$'+str(i))


    #for l in latentsr[label]:
     #   ax[i].fill_between(range(0,96), l, lmean, color=markers[i], alpha=0.1)
        #ax[i].set_xlabel()
        
plt.gcf()



In [ ]:


# tf.keras.models.save_model(model, '_TFmodel')
tf.keras.models.save_model(symencoder, '_TFencoderr')
tf.keras.models.save_model(nencoder, '_TFencodertau')
# tf.keras.models.save_model(encoder, '_TFencoder')
tf.keras.models.save_model(decoder, '_TFdecoder')
